<a href="https://colab.research.google.com/github/isemic/rhombixtechnologies_taks/blob/main/Copy_of_Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installed kaggle using this command as i am going to use API of that dataset instead of downloading a big data**

In [3]:
!pip install Kaggle


In [4]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [5]:
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other


In [6]:
from zipfile import ZipFile
dataset="/content/sentiment140.zip"
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('done')

done


In [7]:
!pip install nltk

**Importing Required Dependencies**

In [8]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score


In [9]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [10]:
print(stopwords.words('english'))

['a', 'about', 'above', 'after', 'again', 'against', 'ain', 'all', 'am', 'an', 'and', 'any', 'are', 'aren', "aren't", 'as', 'at', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'can', 'couldn', "couldn't", 'd', 'did', 'didn', "didn't", 'do', 'does', 'doesn', "doesn't", 'doing', 'don', "don't", 'down', 'during', 'each', 'few', 'for', 'from', 'further', 'had', 'hadn', "hadn't", 'has', 'hasn', "hasn't", 'have', 'haven', "haven't", 'having', 'he', "he'd", "he'll", 'her', 'here', 'hers', 'herself', "he's", 'him', 'himself', 'his', 'how', 'i', "i'd", 'if', "i'll", "i'm", 'in', 'into', 'is', 'isn', "isn't", 'it', "it'd", "it'll", "it's", 'its', 'itself', "i've", 'just', 'll', 'm', 'ma', 'me', 'mightn', "mightn't", 'more', 'most', 'mustn', "mustn't", 'my', 'myself', 'needn', "needn't", 'no', 'nor', 'not', 'now', 'o', 'of', 'off', 'on', 'once', 'only', 'or', 'other', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 're', 's', 'same', 'shan', "shan't", 'she

Loading the dataset and taking some samples from this data for smoothless working

In [11]:
import pandas as pd
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')
twitter_data1=twitter_data.sample(n=100000,random_state=42)
twitter_data.shape

(1599999, 6)

In [12]:
twitter_data1.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick"


**Adding Column Names as the dataset Doesn't have that. **

In [13]:
column_names=['target','ids','date','flag','user','text']
twitter_data1.columns=column_names
twitter_data1.head()

,target,ids,date,flag,user,text
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick"


In [14]:
twitter_data1.isnull().sum()# checking for null values

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [15]:
twitter_data1.dtypes#checking the data type

,0
target,int64
ids,int64
date,object
flag,object
user,object
text,object


In [16]:
twitter_data1['target'].value_counts()

,count
target,
4,50057
0,49943


In [17]:
twitter_data1.replace({'target':{4:1}},inplace=True) #replacing 4 by 1 in the dataset

In [18]:
twitter_data1['target'].value_counts() #rechecking values

,count
target,
1,50057
0,49943


In [19]:
port=PorterStemmer()


In [20]:
#defining a fuction for getting the stopwords out of the original text.
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]',' ',content)
  stemmed_content=stemmed_content.lower()
  stemmed_content=stemmed_content.split()
  stemmed_content=[port.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content=' '.join(stemmed_content)
  return stemmed_content


In [21]:
twitter_data1['stemmed_text']=twitter_data1['text'].apply(stemming) #Applying the stemming on the data

In [22]:
twitter_data1.head()

,target,ids,date,flag,user,text,stemmed_text
541200,0,2200003313,Tue Jun 16 18:18:13 PDT 2009,NO_QUERY,DEWGetMeTho77,@Nkluvr4eva My poor little dumpling In Holmde...,nkluvr eva poor littl dumpl holmdel vid realli...
750,0,1467998601,Mon Apr 06 23:11:18 PDT 2009,NO_QUERY,Young_J,I'm off too bed. I gotta wake up hella early t...,bed gotta wake hella earli tomorrow morn
766711,0,2300049112,Tue Jun 23 13:40:12 PDT 2009,NO_QUERY,dougnawoschik,I havent been able to listen to it yet My spe...,havent abl listen yet speaker bust
285055,0,1993474319,Mon Jun 01 10:26:09 PDT 2009,NO_QUERY,thireven,now remembers why solving a relatively big equ...,rememb solv rel big equat two unknown total pa...
705995,0,2256551006,Sat Jun 20 12:56:51 PDT 2009,NO_QUERY,taracollins086,"Ate too much, feel sick",ate much feel sick


**seperating the columns values into x and y **

In [23]:
x=twitter_data1['stemmed_text'].values
y=twitter_data1['target'].values

In [24]:
print(x)
print(y)

['nkluvr eva poor littl dumpl holmdel vid realli tri hope dont tri hard tonight xx'
 'bed gotta wake hella earli tomorrow morn'
 'havent abl listen yet speaker bust' ... 'go watch tv hope tivo someth'
 'good night yesterday wish neighbour children wake earli though'
 'sunday day rest mani other anoth day work what wrong pictur']
[0 0 0 ... 1 0 0]


*Using train test and split for further using it into model***

In [25]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)
print(x.shape,x_train.shape,x_test.shape)

(100000,) (80000,) (20000,)


In [26]:
print(x_test)

['lose nail whit tabl' 'think http plurk com p kqi'
 'tire sleep last night morn today might crash soon must make kmaaaaaaaaaaaart'
 ... 'musicishealthi poor morn guy ring door want read bibl said'
 'loos drive what point' 'yoli lol definit']


**MAIN PART  using vectorizer to transform textual or categorical data into numerical data for easy and efficient computing **

In [27]:
vectorizer=TfidfVectorizer()
x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)


In [28]:
print(x_test)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 135256 stored elements and shape (20000, 66684)>
  Coords	Values
  (0, 34488)	0.4175834452181089
  (0, 40285)	0.4553664454758065
  (0, 56699)	0.48388262000430077
  (0, 63814)	0.6197766345001054
  (1, 11286)	0.44405025882627075
  (1, 25247)	0.4129587506055886
  (1, 45553)	0.6577939900436636
  (1, 58503)	0.4467566503109159
  (2, 12136)	0.415985702626823
  (2, 32641)	0.2676346827241616
  (2, 35601)	0.25572265775284003
  (2, 37732)	0.338382619143351
  (2, 39087)	0.27098388427628367
  (2, 39916)	0.3474204546589057
  (2, 41318)	0.2569587458738838
  (2, 53364)	0.2747627945351066
  (2, 54119)	0.3043827927274357
  (2, 59068)	0.3057442681218447
  (2, 59275)	0.23161741513564174
  (3, 41318)	0.42659654049881496
  (3, 63490)	0.5923308276896359
  (3, 63495)	0.42501068025255956
  (3, 63547)	0.53528077106712
  (4, 7502)	0.7399409413493427
  (4, 26935)	0.6726718392463361
  :	:
  (19995, 55447)	0.2553875489295339
  (19995, 58503)	0.1885962111

**logistic Regression model as the outcome has only two options in my case**

In [29]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(x_train,y_train)

LogisticRegression()

In [30]:
predicted=model.predict(x_train)
accuracy=accuracy_score(predicted,y_train)
print(accuracy*100)

82.80625


In [31]:
predicted=model.predict(x_test)
accuracy=accuracy_score(y_test,predicted)
print(accuracy*100)

75.86500000000001


In [32]:
import pickle
file='trained_model.sav'

pickle.dump(model,open(file,'wb'))
pickle.dump(vectorizer,open('transform.pkl','wb'))

In [33]:
# Sample tweet or text input
sample_text = "you make me happy"

# Transform and predict
transformed = vectorizer.transform([sample_text])
prediction = model.predict(transformed)

print("Prediction:", prediction[0])


Prediction: 1
